# Experiment 10-12
- **Model:** `yolov8n` *(Nano)*
- **Dataset:**
    1. 5m | 60º
    1. 3m + 5m | 60º
- **Sizes:** large
- **Experiments:**
    1. large


## Init

In [1]:
import os
import shutil
import fnmatch

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 20.4 MB/s eta 0:00:00


## Helper Functions

In [3]:
# Change for different file formats
reference = {
  "small": {
    "suffix": ".S",
    "file": "209"
  },
  "mid": {
    "suffix": ".M",
    "file": "503"
  },
  "large": {
    "suffix": ".L",
    "file": "000"
  }
}

In [4]:
# Clone config files
def copy_config(src_folder, dest_folder):
    """
    Copies files from src_folder to dest_folder, excluding subfolders.

    Args:
        src_folder: The path to the source folder.
        dest_folder: The path to the destination folder.
    """

    try:
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_path):
                shutil.copy2(src_path, dest_path) #copy metadata as well.
                #Use shutil.copy for not copying metadata.
                print(f"Copied: {filename}")
            #else: #optional
                #print(f"Skipped (not a file): {filename}") #optional. Uncomment if you want to see the skipped folders.

        print("✅ Copying complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [5]:
# Copy filtered dataset images/labels
def copy_and_filter_folder(src_folder, dest_folder, pattern):
    """
    Copies a folder and files that match the given pattern.
    Alerts the user when a folder or file already exists but *does not* overwrite.
    Creates only what is needed.

    :param src_folder: Path to the source folder.
    :param dest_folder: Path to the destination folder.
    :param pattern: Filename pattern to keep (e.g., "*.txt").
    """
    try:
        # Ensure destination folder exists
        if not os.path.exists(dest_folder):
            print(f"✓ Creating destination folder '{dest_folder}'.\n")
            os.makedirs(dest_folder)
        else:
            print(f"✓ Destination folder '{dest_folder}' already exists.\n")

        # Walk through the source folder
        for root, _, files in os.walk(src_folder):
            relative_path = os.path.relpath(root, src_folder)
            new_root = os.path.join(dest_folder, relative_path)

            if not os.path.exists(new_root):
                print(f"Creating subdirectory '{new_root}'")
                os.makedirs(new_root)
            else:
                print(f"❕Subdirectory '{new_root}' already exists.")
                print("Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.")

            for file in files:
                if fnmatch.fnmatch(file, pattern + "*"):
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(new_root, file)

                    if not os.path.exists(dest_file):
                        shutil.copy2(src_file, dest_file)  # copy metadata as well
                    else:
                        print(f"❗️File '{dest_file}' already exists. Skipping.")

            print(f" ✓ Copying files complete.\n")
        print("✅ Copying dataset complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [6]:
def save_on_cloud(source: str, destination: str):
    """
    Saves a folder to a cloud storage location (e.g., Google Drive in Colab).

    Args:
        source (str): The path to the source folder.
        destination (str): The path to the destination folder (in cloud storage).
    """
    # 0. Input Validation (Assertions)
    assert isinstance(source, str), "Source must be a string."
    assert isinstance(destination, str), "Destination must be a string."

    try:
        # 1. Verify Source Folder
        if not os.path.exists(destination):
            os.makedirs(destination)

        # 2. Copy the Folder
        shutil.copytree(source, destination, dirs_exist_ok=True)
        print("✅ Folder copied successfully:\n  ",source,"\n  -->",destination)

    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Datasets builder

## Importing from Drive

In [7]:
!rm -rf /content/sample_data

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Check if the cloud path is ok and the dataset can be found
!ls /content/drive/MyDrive/YOLO

3.5m.v3i.yolov8  3m60.v2i.yolov8  5m60.v1i.yolov8  5m.v1i.yolov8


In [10]:
drive_path = '/content/drive/MyDrive/YOLO'
drive_datasets_paths = os.listdir(drive_path)
drive_datasets = len(drive_datasets_paths)
if (drive_datasets) > 1:
    print("There are %d dataset options:" % drive_datasets)
else:
    print("Theres is only 1 dataset:")
drive_datasets_paths

There are 4 dataset options:


['3.5m.v3i.yolov8', '5m.v1i.yolov8', '5m60.v1i.yolov8', '3m60.v2i.yolov8']

In [14]:
choose_dataset = 3
index = choose_dataset - 1
model_name = os.listdir(drive_path)[index]
print("Choosed model:", model_name)

Choosed model: 5m60.v1i.yolov8


***Readme:***
*   **Option 1:** is desirable if you need to test many subset combinations in the same session (avoid downloading data twice from the cloud).
*   **Option 3:** is best if you're just going to test one subset combination  (avoid downloading unnecessary data from the cloud).



In [15]:
# Option 1 (download full dataset)
source = f"/content/drive/MyDrive/YOLO/{model_name}"
dest = f"/content/YOLO/{model_name}"
!mkdir /content/YOLO/
!cp -r $source $dest
src_folder = dest

mkdir: cannot create directory ‘/content/YOLO/’: File exists


In [13]:
# Option 3 (download just what's needed)
# src_folder = f"{drive_path}/{model}"

In [47]:
!rm -rf "/content/YOLO/3.5m.v3i.yolov8"

In [24]:
!mkdir "/content/YOLO/5m60.v1i.yolov8.ML"
!mkdir "/content/other"
!cp -r "/content/YOLO/5m60.v1i.yolov8/" "/content/other"

In [27]:

!mv "/content/other/5m60.v1i.yolov8.ML" "/content/YOLO/"

## Merging datasets

In [38]:
import shutil
import os

def copy_directory(source, destination, overwrite=True):
    try:
        if overwrite and os.path.exists(destination):
            shutil.rmtree(destination)  # Remove existing directory
        shutil.copytree(source, destination)
        print(f"Directory '{source}' copied to '{destination}'")
        return True  # Indicate success
    except FileExistsError:
        print(f"Destination '{destination}' exists. Use overwrite=True to replace.")
        return False  # Indicate failure

In [39]:
source = "/content/YOLO/3m60.v2i.yolov8/train/images"
dest = "/content/YOLO/5m60.v1i.yolov8/train/images"

In [40]:
copy_directory(source, dest, overwrite=True)

Directory '/content/YOLO/3m60.v2i.yolov8/train/images' copied to '/content/YOLO/5m60.v1i.yolov8/train/images'


True

In [41]:
source = "/content/YOLO/3m60.v2i.yolov8/train/labels"
dest = "/content/YOLO/5m60.v1i.yolov8/train/labels"

In [42]:
copy_directory(source, dest, overwrite=True)

Directory '/content/YOLO/3m60.v2i.yolov8/train/labels' copied to '/content/YOLO/5m60.v1i.yolov8/train/labels'


True

In [43]:
source = "/content/YOLO/3m60.v2i.yolov8/valid/images"
dest = "/content/YOLO/5m60.v1i.yolov8/valid/images"

In [44]:
copy_directory(source, dest, overwrite=True)

Directory '/content/YOLO/3m60.v2i.yolov8/valid/images' copied to '/content/YOLO/5m60.v1i.yolov8/valid/images'


True

In [45]:
source = "/content/YOLO/3m60.v2i.yolov8/valid/labels"
dest = "/content/YOLO/5m60.v1i.yolov8/valid/labels"

In [46]:
copy_directory(source, dest, overwrite=True)

Directory '/content/YOLO/3m60.v2i.yolov8/valid/labels' copied to '/content/YOLO/5m60.v1i.yolov8/valid/labels'


True

## Build dataset 'large'

### Settings

In [ ]:
# CHOOSE THE DATASET YOU WANT TO WORK WITH
choosen_data = 'large' # Options: small / mid / large

In [ ]:
suffix = reference[choosen_data]['suffix']
dest_folder = f"/content/YOLO/{model_name}{suffix}"
pattern = reference[choosen_data]['file']

In [ ]:
print(f'🆗 SETTING PARAMETERS\n - Model: {model_name}\n - rigin: {src_folder}\n - Destination: {dest_folder}\n - Prefix: {pattern}…')

🆗 SETTING PARAMETERS
 - Model: 3.5m.v3i.yolov8
 - rigin: /content/YOLO/3.5m.v3i.yolov8
 - Destination: /content/YOLO/3.5m.v3i.yolov8.S
 - Prefix: 209…


### Builder

In [ ]:
copy_config(src_folder, dest_folder)

Copied: README.roboflow.txt
Copied: README.dataset.txt
Copied: data.yaml
✅ Copying complete.


In [ ]:
copy_and_filter_folder(src_folder, dest_folder, pattern)

✓ Destination folder '/content/YOLO/3.5m.v3i.yolov8.S' already exists.

❕Subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/.' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/valid'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/valid/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/valid/labels'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/train'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/train/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/train/labels'
 ✓ Copying files complete.

✅ Copying dataset complete.


## Build dataset 'medium'

### Settings

In [48]:
# CHOOSE THE DATASET YOU WANT TO WORK WITH
choosen_data = 'mid' # Options: small / mid / large

In [51]:
suffix = reference[choosen_data]['suffix']
#dest_folder = f"/content/YOLO/{model_name}{suffix}"
pattern = reference[choosen_data]['file']

In [52]:
src_folder = "/content/YOLO/3m60.v2i.yolov8"
dest_folder = "/content/YOLO/5m60.v1i.yolov8.ML"

In [53]:
print(f'🆗 SETTING PARAMETERS\n - Model: {model_name}\n - rigin: {src_folder}\n - Destination: {dest_folder}\n - Prefix: {pattern}…')

🆗 SETTING PARAMETERS
 - Model: 5m60.v1i.yolov8
 - rigin: /content/YOLO/3m60.v2i.yolov8
 - Destination: /content/YOLO/5m60.v1i.yolov8.ML
 - Prefix: 503…


### Builder

In [ ]:
copy_config(src_folder, dest_folder)

Copied: README.roboflow.txt
Copied: README.dataset.txt
Copied: data.yaml
✅ Copying complete.


In [54]:
copy_and_filter_folder(src_folder, dest_folder, pattern)

✓ Destination folder '/content/YOLO/5m60.v1i.yolov8.ML' already exists.

❕Subdirectory '/content/YOLO/5m60.v1i.yolov8.ML/.' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

❕Subdirectory '/content/YOLO/5m60.v1i.yolov8.ML/valid' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

❕Subdirectory '/content/YOLO/5m60.v1i.yolov8.ML/valid/images' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

❕Subdirectory '/content/YOLO/5m60.v1i.yolov8.ML/valid/labels' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

❕Subdirectory '/content/YOLO/5m60.v1i.yolov8.ML/train' already exists.
Make sure the data inside is r

## Remove datasets (restart)

In [ ]:
# Removes complete local dataset
#!rm -rf /content/YOLO

## Download model

In [55]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolov8n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 112MB/s]


# Finetuning

### Info

In [56]:
!nvidia-smi

Fri Feb 21 20:13:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [57]:
!yolo version

8.3.78


-----
## Experiment 10 *(full dataset)*
### *5m | 60º | large*

### Train

In [ ]:

# Train the model
model.train(
    data=f"/content/YOLO/{model_name}/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/5m60.v1i.yolov8/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=240, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 16.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[240] must be multiple of max stride 32, updating to [256]


...

                   all        110       1910      0.381      0.349       0.31      0.105



100 epochs completed in 0.060 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


                   all        110       1910      0.376      0.349      0.309      0.105
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782193c81450>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [ ]:
# Load the trained YOLO model
model1 = YOLO("/content/runs/detect/train/weights/best.pt")

# Validate the model
model1.val(data=f"/content/YOLO/{model_name}/data.yaml")

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/5m60.v1i.yolov8/valid/labels.cache... 110 images, 3 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.11it/s]


                   all        110       1910      0.384      0.347       0.31      0.104
Speed: 0.4ms preprocess, 2.5ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782195b8fe90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [ ]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model10(5_60_L)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 3 plant-weeds, 12.3ms
Speed: 1.6ms preprocess, 12.3ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model10(5_60_L)/experiment1
Clase: 0, Confianza: 0.74, BBox: (4.17, 161.65, 135.37, 209.77)
Clase: 0, Confianza: 0.48, BBox: (182.02, 185.11, 239.98, 238.40)
Clase: 0, Confianza: 0.43, BBox: (125.25, 198.79, 240.00, 231.30)


In [ ]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.875 0.85 0.25416666666666665 0.125
0 0.9041666666666667 0.8875 0.19583333333333333 0.13333333333333333
0 0.9625 0.9458333333333333 0.07916666666666666 0.08333333333333333
0 0.7083333333333334 0.8791666666666667 0.30833333333333335 0.06666666666666667
0 0.4375 0.8458333333333333 0.20833333333333334 0.041666666666666664
0 0.4041666666666667 0.825 0.225 0.041666666666666664
0 0.3333333333333333 0.7583333333333333 0.21666666666666667 0.09166666666666666
0 0.2375 0.7541666666666667 0.225 0.041666666666666664
0 0.09583333333333334 0.7291666666666666 0.19166666666666668 0.09166666666666666



#### Test 2 (medium weed)

In [ ]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model10(5_60_L)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 256x256 4 plant-weeds, 7.7ms
Speed: 0.9ms preprocess, 7.7ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model10(5_60_L)/experiment2
Clase: 0, Confianza: 0.34, BBox: (22.60, 0.04, 81.20, 53.77)
Clase: 0, Confianza: 0.32, BBox: (0.00, 3.27, 33.06, 75.81)
Clase: 0, Confianza: 0.30, BBox: (0.15, 163.07, 52.65, 207.14)
Clase: 0, Confianza: 0.27, BBox: (28.14, 0.11, 69.57, 45.58)


In [ ]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.05416666666666667 0.7958333333333333 0.10416666666666667 0.13333333333333333
0 0.0875 0.6625 0.175 0.275
0 0.25 0.575 0.5 0.125
0 0.09166666666666666 0.5583333333333333 0.10833333333333334 0.35833333333333334
0 0.05416666666666667 0.48333333333333334 0.10416666666666667 0.23333333333333334
0 0.9875 0.25416666666666665 0.025 0.175
0 0.029166666666666667 0.1375 0.058333333333333334 0.275
0 0.1375 0.10416666666666667 0.21666666666666667 0.20833333333333334
0 0.15416666666666667 0.175 0.20833333333333334 0.275



### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


In [ ]:
# Store experiments
save_on_cloud(source='/content/model10(5_60_L)/', destination='/content/drive/MyDrive/save.SM/')

✅ Folder copied successfully:
   /content/model10(5_60_L)/ 
  --> /content/drive/MyDrive/save.SM/


-----
## Experiment 11 *(full dataset)*
### *3.5m + 5m | 60º | small + mid + large*

### Train

In [58]:

# Train the model
model.train(
    data=f"/content/YOLO/{model_name}/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/5m60.v1i.yolov8/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=240, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 21.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 89.6MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[240] must be multiple of max stride 32, updating to [256]


...

                   all        736       6429      0.407      0.348      0.298     0.0977



100 epochs completed in 0.194 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:05<00:00,  1.00it/s]


                   all        736       6429      0.415      0.356      0.317      0.105
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ee8ffbf5e90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [59]:
# Load the trained YOLO model
model1 = YOLO("/content/runs/detect/train/weights/best.pt")

# Validate the model
model1.val(data=f"/content/YOLO/{model_name}/data.yaml")

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/5m60.v1i.yolov8/valid/labels.cache... 736 images, 45 backgrounds, 0 corrupt: 100%|██████████| 736/736 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:04<00:00,  9.94it/s]


                   all        736       6429      0.416      0.358      0.317      0.105
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ee91430d6d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [60]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [61]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model11(3+5_60_SML)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 8 plant-weeds, 6.3ms
Speed: 1.2ms preprocess, 6.3ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model11(3+5_60_SML)/experiment1
Clase: 0, Confianza: 0.60, BBox: (48.23, 169.23, 134.25, 211.03)
Clase: 0, Confianza: 0.42, BBox: (179.79, 185.52, 240.00, 223.37)
Clase: 0, Confianza: 0.38, BBox: (51.77, 149.12, 102.87, 167.22)
Clase: 0, Confianza: 0.38, BBox: (0.00, 160.43, 43.13, 180.39)
Clase: 0, Confianza: 0.34, BBox: (0.00, 157.06, 50.27, 183.34)
Clase: 0, Confianza: 0.34, BBox: (131.28, 197.55, 203.88, 213.96)
Clase: 0, Confianza: 0.29, BBox: (185.15, 192.08, 240.00, 227.60)
Clase: 0, Confianza: 0.29, BBox: (207.47, 204.71, 239.89, 239.61)


In [62]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.875 0.85 0.25416666666666665 0.125
0 0.9041666666666667 0.8875 0.19583333333333333 0.13333333333333333
0 0.9625 0.9458333333333333 0.07916666666666666 0.08333333333333333
0 0.7083333333333334 0.8791666666666667 0.30833333333333335 0.06666666666666667
0 0.4375 0.8458333333333333 0.20833333333333334 0.041666666666666664
0 0.4041666666666667 0.825 0.225 0.041666666666666664
0 0.3333333333333333 0.7583333333333333 0.21666666666666667 0.09166666666666666
0 0.2375 0.7541666666666667 0.225 0.041666666666666664
0 0.09583333333333334 0.7291666666666666 0.19166666666666668 0.09166666666666666



#### Test 2 (medium weed)

In [63]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [64]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model11(3+5_60_SML)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 256x256 3 plant-weeds, 6.3ms
Speed: 0.9ms preprocess, 6.3ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model11(3+5_60_SML)/experiment2
Clase: 0, Confianza: 0.45, BBox: (3.31, 0.04, 56.70, 54.34)
Clase: 0, Confianza: 0.39, BBox: (0.00, 197.56, 25.87, 239.95)
Clase: 0, Confianza: 0.37, BBox: (18.69, 0.19, 60.06, 66.99)


In [65]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.05416666666666667 0.7958333333333333 0.10416666666666667 0.13333333333333333
0 0.0875 0.6625 0.175 0.275
0 0.25 0.575 0.5 0.125
0 0.09166666666666666 0.5583333333333333 0.10833333333333334 0.35833333333333334
0 0.05416666666666667 0.48333333333333334 0.10416666666666667 0.23333333333333334
0 0.9875 0.25416666666666665 0.025 0.175
0 0.029166666666666667 0.1375 0.058333333333333334 0.275
0 0.1375 0.10416666666666667 0.21666666666666667 0.20833333333333334
0 0.15416666666666667 0.175 0.20833333333333334 0.275



### Save results

In [66]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


In [67]:
# Store experiments
save_on_cloud(source='/content/model11(3+5_60_SML)/', destination='/content/drive/MyDrive/save.SM/')

✅ Folder copied successfully:
   /content/model11(3+5_60_SML)/ 
  --> /content/drive/MyDrive/save.SM/


-----
## Experiment 12 *(full dataset)*
### *3.5m + 5m | 60º | mid + large*

### Train

In [70]:

# Train the model
model.train(
    data=f"/content/YOLO/5m60.v1i.yolov8.ML/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/5m60.v1i.yolov8.ML/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=240, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning /content/YOLO/5m60.v1i.yolov8.ML/train/labels... 557 images, 9 backgrounds, 0 corrupt: 100%|██████████| 557/557 [00:01<00:00, 518.88it/s]

train: New cache created: /content/YOLO/5m60.v1i.yolov8.ML/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/YOLO/5m60.v1i.yolov8.ML/valid/labels... 478 images, 14 backgrounds, 0 corrupt: 100%|██████████| 478/478 [00:00<00:00, 485.18it/s]


val: New cache created: /content/YOLO/5m60.v1i.yolov8.ML/valid/labels.cache
Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        478       6818      0.434      0.363      0.344      0.119



100 epochs completed in 0.151 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.34s/it]


                   all        478       6818      0.446      0.362      0.344      0.119
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ee9143a3a90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [72]:
# Load the trained YOLO model
model2 = YOLO("/content/runs/detect/train3/weights/best.pt")

# Validate the model
model2.val(data=f"/content/YOLO/5m60.v1i.yolov8.ML/data.yaml")

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/5m60.v1i.yolov8.ML/valid/labels.cache... 478 images, 14 backgrounds, 0 corrupt: 100%|██████████| 478/478 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:06<00:00,  4.53it/s]


                   all        478       6818      0.445      0.363      0.344      0.119
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ee8f8916010>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [73]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [74]:
# Ejecutar predicción sobre la imagen
results = model2(image, save=True, project="model12(3+5_60_ML)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 10 plant-weeds, 6.7ms
Speed: 1.3ms preprocess, 6.7ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model12(3+5_60_ML)/experiment1
Clase: 0, Confianza: 0.60, BBox: (179.82, 184.65, 239.88, 236.43)
Clase: 0, Confianza: 0.44, BBox: (0.00, 168.34, 136.54, 211.90)
Clase: 0, Confianza: 0.39, BBox: (183.59, 202.18, 240.00, 240.00)
Clase: 0, Confianza: 0.38, BBox: (131.03, 200.50, 220.42, 224.62)
Clase: 0, Confianza: 0.37, BBox: (28.10, 161.06, 131.24, 210.07)
Clase: 0, Confianza: 0.35, BBox: (178.27, 188.35, 239.70, 224.82)
Clase: 0, Confianza: 0.27, BBox: (199.49, 189.96, 239.91, 238.35)
Clase: 0, Confianza: 0.26, BBox: (140.60, 206.76, 234.08, 239.95)
Clase: 0, Confianza: 0.26, BBox: (0.00, 158.08, 129.03, 204.41)
Clase: 0, Confianza: 0.26, BBox: (51.91, 147.54, 102.02, 168.63)


In [75]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.875 0.85 0.25416666666666665 0.125
0 0.9041666666666667 0.8875 0.19583333333333333 0.13333333333333333
0 0.9625 0.9458333333333333 0.07916666666666666 0.08333333333333333
0 0.7083333333333334 0.8791666666666667 0.30833333333333335 0.06666666666666667
0 0.4375 0.8458333333333333 0.20833333333333334 0.041666666666666664
0 0.4041666666666667 0.825 0.225 0.041666666666666664
0 0.3333333333333333 0.7583333333333333 0.21666666666666667 0.09166666666666666
0 0.2375 0.7541666666666667 0.225 0.041666666666666664
0 0.09583333333333334 0.7291666666666666 0.19166666666666668 0.09166666666666666



#### Test 2 (medium weed)

In [76]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [77]:
# Ejecutar predicción sobre la imagen
results = model2(image, save=True, project="model12(3+5_60_ML)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 256x256 6 plant-weeds, 6.5ms
Speed: 0.9ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model12(3+5_60_ML)/experiment2
Clase: 0, Confianza: 0.43, BBox: (0.25, 199.93, 25.62, 240.00)
Clase: 0, Confianza: 0.35, BBox: (4.05, 0.43, 52.20, 49.03)
Clase: 0, Confianza: 0.32, BBox: (20.94, 125.20, 150.54, 150.08)
Clase: 0, Confianza: 0.30, BBox: (21.55, 0.12, 76.58, 51.17)
Clase: 0, Confianza: 0.28, BBox: (21.49, 2.26, 59.78, 53.82)
Clase: 0, Confianza: 0.25, BBox: (0.09, 26.66, 28.54, 71.23)


In [78]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.05416666666666667 0.7958333333333333 0.10416666666666667 0.13333333333333333
0 0.0875 0.6625 0.175 0.275
0 0.25 0.575 0.5 0.125
0 0.09166666666666666 0.5583333333333333 0.10833333333333334 0.35833333333333334
0 0.05416666666666667 0.48333333333333334 0.10416666666666667 0.23333333333333334
0 0.9875 0.25416666666666665 0.025 0.175
0 0.029166666666666667 0.1375 0.058333333333333334 0.275
0 0.1375 0.10416666666666667 0.21666666666666667 0.20833333333333334
0 0.15416666666666667 0.175 0.20833333333333334 0.275



### Save results

In [81]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


In [82]:
# Store experiments
save_on_cloud(source='/content/model12(3+5_60_ML)/', destination='/content/drive/MyDrive/save.ML/')

✅ Folder copied successfully:
   /content/model12(3+5_60_ML)/ 
  --> /content/drive/MyDrive/save.ML/
